In [ ]:
import os
import re
import glob
import itertools
import numpy as np
import os.path as op

In [ ]:
paths_data = op.join("/path", "to", "data")
paths_save = op.join("paths", "to", "figure11")
os.makedirs(paths_save, exist_ok = True)

In [ ]:
fname_list = glob.glob(op.join(paths_data, "**", "**", "*.npy"))
n_labels   = np.load(fname_list[0]).shape[0] # number of labels

dataset_list = set([re.sub(".+((multi|single)-shell).+", "\\1", x) for x in fname_list])
method_list  = set([re.sub(".+_method-(\w+)_.+", "\\1", x) for x in fname_list])
repeat_list  = set([re.sub(".+_repeat-(\d+)_.+", "\\1", x) for x in fname_list])
suffix_list  = ["ytrue", "ypred", "yprob"] 

conds = itertools.product(dataset_list, method_list, suffix_list)
for dataset, method, suffix in conds: # for each dataset and method
  # filter the file names by dataset and method
  cond_list = [x for x in fname_list if dataset in x and method in x]
  
  # load the current values
  save_name   = f"figure11_{dataset}_method-{method}_{suffix}.npy"
  curr_list   = [x for x in cond_list if suffix in x]
  
  # initialize the current matrix
  if suffix == "yprob": # if the suffix is yprob, add dimension
    curr_matrix = np.ones((n_labels, 5, len(repeat_list))) * np.nan
  else: # else, (n_labels, n_repeats)
    curr_matrix = np.ones((n_labels, len(repeat_list))) * np.nan
    
  # fill the current matrix with values
  for fname in curr_list: # for each file name
    repeat = int(re.sub(".+_repeat-(\d+)_.+", "\\1", fname)) - 1 # 0-based
    curr_matrix[..., repeat] = np.load(fname)
  
  # save the current values
  np.save(op.join(paths_save, save_name), curr_matrix)
  print(f"Saved: {save_name}")